<a href="https://colab.research.google.com/github/Jakelinecs/Tareas-Machine-Learning/blob/main/N26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
TensorFlow neural network implementation for Iris dataset (3-class) multi-class classification
Using TensorFlow 1.x API
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.datasets import load_iris
import tensorflow as tf

# TensorFlow 1.x環境で実行する場合は、tf.compat.v1を使用してレガシーなAPIを使用します
try:
    tf.compat.v1.disable_eager_execution()
except:
    pass  # Si ya está deshabilitado o no es necesario

# データセットの読み込み (Iris dataset from sklearn)
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['Species'] = iris.target_names[iris.target]

# 3種類すべての目的変数を使用
y_labels = df["Species"]
X = df[["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]]

# NumPy 配列に変換
X = np.array(X)
y_labels = np.array(y_labels)

# ラベルをOne-Hotエンコーディングに変換 (3クラス分類のため)
lb = LabelBinarizer()
y = lb.fit_transform(y_labels)
# yのshapeは (n_samples, 3) となる

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ (元のコードから変更なし)
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        # NumPyの整数型キャストをnp.int32に修正
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int32)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = y.shape[1] # クラス数を3に変更

# 計算グラフに渡す引数の形を決める
X = tf.compat.v1.placeholder("float", [None, n_input], name="X_input")
Y = tf.compat.v1.placeholder("float", [None, n_classes], name="Y_labels") # n_classesを3に変更

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク (最終層の出力サイズをn_classes=3に変更)
    """
    tf.compat.v1.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.compat.v1.Variable(tf.random.normal([n_input, n_hidden1]), name='W1'),
        'w2': tf.compat.v1.Variable(tf.random.normal([n_hidden1, n_hidden2]), name='W2'),
        'w3': tf.compat.v1.Variable(tf.random.normal([n_hidden2, n_classes]), name='W3') # 出力ノード数をn_classes=3に変更
    }
    biases = {
        'b1': tf.compat.v1.Variable(tf.random.normal([n_hidden1]), name='b1'),
        'b2': tf.compat.v1.Variable(tf.random.normal([n_hidden2]), name='b2'),
        'b3': tf.compat.v1.Variable(tf.random.normal([n_classes]), name='b3') # バイアスサイズもn_classes=3に変更
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み
logits = example_net(X)

# 目的関数
# ★変更点1: tf.nn.sigmoid_cross_entropy_with_logitsを多クラス用のtf.nn.softmax_cross_entropy_with_logitsに変更
# TensorFlow 2.xではsoftmax_cross_entropy_with_logits_v2が廃止され、softmax_cross_entropy_with_logitsを使用
# labelsはOne-Hotベクトル、logitsは最終出力層の線形結合値
try:
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
except:
    # Fallback para versiones más antiguas
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果と指標値計算
# ★変更点2: 2値分類のtf.signを使った判定を、多クラス用の最大値インデックス比較に変更
# tf.argmaxで最大値を持つインデックス（予測クラス）を取得
prediction = tf.argmax(logits, 1)
# tf.argmaxで最大値を持つインデックス（正解クラス）を取得
correct_label = tf.argmax(Y, 1)
# 予測クラスと正解クラスが一致しているか比較
correct_pred = tf.equal(prediction, correct_label)
# 正しい予測の平均を計算して精度とする
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.compat.v1.global_variables_initializer()


# 計算グラフの実行
with tf.compat.v1.Session() as sess:
    sess.run(init)
    print("--- 3-class classification (Iris) training started ---")
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int32)
        total_loss = 0

        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            # 訓練オペレーションの実行 (重み更新)
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            # 損失計算
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss

        total_loss /= X_train.shape[0] # サンプル数で割って平均損失を出す

        # 検証データでの損失と精度を計算
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})

        print("Epoch {:03d}, loss : {:.4f}, val_loss : {:.4f}, val_acc : {:.3f}".format(
            epoch + 1, total_loss, val_loss, val_acc))

    # テストデータでの精度を計算
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("--- Training completed ---")
    print("test_acc : {:.3f}".format(test_acc))


--- 3-class classification (Iris) training started ---
Epoch 001, loss : 17.1048, val_loss : 145.4677, val_acc : 0.500
Epoch 002, loss : 13.6411, val_loss : 118.4701, val_acc : 0.708
Epoch 003, loss : 10.3908, val_loss : 87.9416, val_acc : 0.583
Epoch 004, loss : 7.1911, val_loss : 59.1567, val_acc : 0.708
Epoch 005, loss : 3.8212, val_loss : 27.8530, val_acc : 0.667
Epoch 006, loss : 0.9779, val_loss : 10.3551, val_acc : 0.292
Epoch 007, loss : 0.6544, val_loss : 7.4630, val_acc : 0.625
Epoch 008, loss : 0.4310, val_loss : 7.6317, val_acc : 0.667
Epoch 009, loss : 0.2822, val_loss : 4.9857, val_acc : 0.542
Epoch 010, loss : 0.2453, val_loss : 6.3684, val_acc : 0.667
Epoch 011, loss : 0.1873, val_loss : 4.6357, val_acc : 0.625
Epoch 012, loss : 0.1830, val_loss : 4.6036, val_acc : 0.625
Epoch 013, loss : 0.1638, val_loss : 3.9203, val_acc : 0.625
Epoch 014, loss : 0.1548, val_loss : 3.6022, val_acc : 0.667
Epoch 015, loss : 0.1364, val_loss : 3.0378, val_acc : 0.708
Epoch 016, loss : 0